In [12]:
import networkx as nx
import csv
import time
from collections import defaultdict
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random


In [13]:
class Service:
    def __init__(
        self,
        service_id, 
        rake_num,
        start_station,
        start_time,
        end_station,
        end_time,
        direction,
        service_time=0,
        same_jurisdiction=None,
        step_back_rake=None,
        step_back_location=None,
        merged_rake_num1=None,
        merged_rake_num2=None
    ):
        self.service_id = str(service_id)
        self.rake_num = rake_num
        self.start_station = start_station
        self.start_time = start_time
        self.end_station = end_station
        self.end_time = end_time
        self.direction = direction
        self.service_time = int(service_time) if service_time else 0
        self.same_jurisdiction = same_jurisdiction
        self.step_back_rake = step_back_rake
        self.step_back_location = step_back_location
        self.merged_rake_num1 = merged_rake_num1
        self.merged_rake_num2 = merged_rake_num2


def load_services(csv_file):
    df = pd.read_csv(csv_file)
    services = []
    for _, row in df.iterrows():
        s = Service(
            service_id=row.get("Service"),   
            rake_num=row.get("Rake Num"),
            start_station=get_base_station_name(row.get("Start Station")),
            start_time=parse_time_to_minutes(row.get("Start Time")),
            end_station=get_base_station_name(row.get("End Station")),
            end_time=parse_time_to_minutes(row.get("End Time")),
            direction=row.get("Direction"),
            service_time=int(row["service time"]) if "service time" in row and pd.notna(row["service time"]) else 0,
            same_jurisdiction=row["Same Jurisdiction"] if "Same Jurisdiction" in row else None,
            step_back_rake=row["Step Back Rake"] if "Step Back Rake" in row else None,
            step_back_location=row["Step Back Location"] if "Step Back Location" in row else None,
            merged_rake_num1=row["mergedRakeNum1"] if "mergedRakeNum1" in row else None,
            merged_rake_num2=row["mergedRakeNum2"] if "mergedRakeNum2" in row else None,
        )
        services.append(s)
    return services


#----------------------------------# Adding Dummy Nodes
# Dummy start node
start_service = Service(
    service_id="S",
    rake_num=None,
    start_station=None,
    start_time=None,
    end_station=None,
    end_time=None,
    direction=None,
    service_time=0,
    same_jurisdiction=None,
    step_back_rake=None,
    step_back_location=None,
    merged_rake_num1=None,
    merged_rake_num2=None
)

# Dummy end node
end_service = Service(
    service_id="T",
    rake_num=None,
    start_station=None,
    start_time=None,
    end_station=None,
    end_time=None,
    direction=None,
    service_time=0,
    same_jurisdiction=None,
    step_back_rake=None,
    step_back_location=None,
    merged_rake_num1=None,
    merged_rake_num2=None
)


In [14]:
# === Configuration ===
TIMETABLE_FILE = "C:/Users/srupe/Downloads/crew/mainLoop_rupesh.csv"
MIN_RAKE_GAP_MINUTES = 30   # Minimum required gap between different rakes
ALLOWED_RAKE_CHANGE_STATIONS = {"KKDA", "PVGW"}  # Allowed rake-change stations
MAX_CONNECTION_GAP_MINUTES = 120   # Maximum allowed gap between services
BREAK_STATIONS = {"KKDA", "PVGW"}  # Breaks allowed only here

# === Jurisdiction Buckets ===
jurisdiction_dict = {
    1: {'MKPR','MKPR UP','MKPR DN','SAKP','DDSC','DDSC DN PF','DDSC SDG','DDSC SDG STABLE (DAY)','DDSC DN',
        'DDSC SDG','PVGW','PVGW UP','PVGW DN','MKPD','MKPD','SAKP 3RD','SAKP 3RD PF','MKPR DN SDG','MKPR DN PF','DDSC DN SDG'},
    2: {'MUPR DN SDG STABLE (DAY)','MUPR 4TH SDG STABLE (DAY)','MUPR 3RD SDG STABLE','SVVR','SVVR DN','MUPR',
        'MUPR DN','MUPR 4TH','MUPR 3RD SDG','KKDA DN','KKDA UP','IPE','IPE 3RD PF','IPE 3RD','VND','VND (M)',
        'MVPO','MVPO DN','NZM','NIZM','KKDA','MUPR DN SDG','MVPO DN PF','SVVR DN PF','MUPR 3RD SDG','MUPR 4TH PF',
        'MUPR 4TH SDG','MUPR DN PF','MUPR DN SDG','MUPR DN SDG'}
}


# === Utility Functions ===
def parse_time_to_minutes(t: str):
    """Converts a time string 'HH:MM' into minutes since midnight."""
    try:
        t = t.strip()
        hours, minutes = map(int, t.split(":"))
        return hours * 60 + minutes
    except:
        return None


def get_base_station_name(station: str):
    """Extracts the base station name (first word) in uppercase."""
    if not station:
        return None
    return station.strip().split()[0].upper()


def rake_feasible_connection(s1, s2):
    """
    Checks if a connection between two services is feasible.
    - Must be same base station.
    - 0 <= (start - end) <= MAX_CONNECTION_GAP_MINUTES
    - If rake is same: allow directly.
    - If rake is different: allowed only at ALLOWED_RAKE_CHANGE_STATIONS 
      and gap >= MIN_RAKE_GAP_MINUTES.
    """
    # Handle dummy start/end services
    if not s1.end_station or not s2.start_station:
        return False

    if s1.end_station != s2.start_station:
        return False

    gap = s2.start_time - s1.end_time

    # Must be non-negative and within max connection window
    if gap < 0 or gap > MAX_CONNECTION_GAP_MINUTES:
        return False

    if s1.rake_num == s2.rake_num:
        # Same rake: gap condition is already checked
        return True
    else:
        # Rake change allowed only at certain stations with min gap
        return (s1.end_station in ALLOWED_RAKE_CHANGE_STATIONS) and (gap >= MIN_RAKE_GAP_MINUTES)


## Graph building

In [15]:
def build_graph(services_list, start_service, end_service):
    """
    Build a directed graph from Service objects.
    All nodes are Service objects (including start and end dummies).
    """
    G = nx.DiGraph()

    # Add nodes (store Service object directly as key)
    for s in services_list:
        G.add_node(s.service_id, data=s)

    # Add start/end dummy nodes
    G.add_node(start_service.service_id, data=start_service)
    G.add_node(end_service.service_id, data=end_service)

    # Connect source to all services
    for s in services_list:
        G.add_edge(start_service.service_id, s.service_id, color="black")

    # Feasible service_id-to-service_id connections
    for i, s1 in enumerate(services_list):
        for j, s2 in enumerate(services_list):
            if i == j:
                continue
            if rake_feasible_connection(s1, s2):
                #edge_color = "red" if s1.rake_num != s2.rake_num else "gray"
                G.add_edge(s1.service_id, s2.service_id)

    # Connect all services to sink
    for s in services_list:
        G.add_edge(s.service_id, end_service.service_id, color="black")

    return G


## Checking Current path

In [16]:
def is_path_acceptable(
    path, end_service,
    total_driving_time, driving_time_limit,
    duty_time_limit
):
    """
    Checks whether a partial path is acceptable:
    - Driving time limit
    - Duty time limit
    - Continuous driving 180-min rule
    """
# # Don’t waste time checking tiny paths — they are always okay. Let them grow


#     # Step 0: Skip very short paths (no need for further checks)
#     if len(path) <= 4:
#         return True
    
    # -----------------------------
    # Condition 1: Driving time limit
    # -----------------------------
    if total_driving_time > driving_time_limit:
        return False

    # -----------------------------
    # Condition 2: Duty time limit
    # -----------------------------
    first_service_start_time = path[1].start_time  # skip dummy SOURCE

    if path[-1] == end_service:
        last_service = path[-2]  # path ended with sink dummy
    else:
        last_service = path[-1]

    last_service_end_time = last_service.end_time
    total_duty_time = last_service_end_time - first_service_start_time

    # Morning/evening tighter limit
    if first_service_start_time < 360 or first_service_start_time > 1410:
        effective_duty_limit = 405
    else:
        effective_duty_limit = duty_time_limit  # usually 445

    if total_duty_time > effective_duty_limit:
        return False

    # -----------------------------
    # Condition 3: Continuous driving 180-min rule
    # -----------------------------
    continuous_drive = 0
    # skip dummy SOURCE and SINK
    end_index = len(path) - 1 if path[-1] == end_service else len(path)

    for i in range(1, end_index - 1):
        current_service = path[i]
        next_service = path[i + 1]

        service_time = current_service.service_time or 0
        continuous_drive += service_time

        gap = next_service.start_time - current_service.end_time

        # Rule 1: cannot exceed 180
        if continuous_drive > 180:
            return False

        # Rule 2: if 120 ≤ continuous_drive ≤ 180 then gap must be ≥ 50
        if 120 <= continuous_drive <= 180:
            if gap < 50:
                return False
            else:
                continuous_drive = 0  # reset after required long break

        # Rule 3: normal break → gap > 30 resets counter
        elif gap > 30:
            continuous_drive = 0

    # -----------------------------
    # All conditions passed
    # -----------------------------
    return True


## Checking Final Path

In [17]:
def is_final_path_valid(path):
    """
    Check if a completed path ending at end_service is valid based on:
    1. Jurisdiction overlap between first and last duty.
    2. Required break conditions.
    """

    # === Jurisdiction check ===
    start_station_first_duty = path[1].start_station
    end_station_last_duty = path[-2].end_station

    start_groups = get_jurisdiction_groups(start_station_first_duty)
    end_groups = get_jurisdiction_groups(end_station_last_duty)

    # If no overlap in jurisdiction groups = invalid path
    if start_groups.isdisjoint(end_groups):
        return False

    # === Break check ===
    if not has_required_breaks(path):
        return False

    return True


def get_jurisdiction_groups(station):
    """
    Return the set of jurisdiction groups a station belongs to.
    Uses global jurisdiction_dict.
    """
    return {
        Jurisdiction_group_id
        for Jurisdiction_group_id, stations in jurisdiction_dict.items()
        if station in stations
    }


def has_required_breaks(path):
    """
    Check break conditions:
    Case 1: At least one 30min break AND one 50min break, cumulative < 120
    Case 2: Only one break >=50min, and <120
    Additional:
    - At least one break must lie within the jurisdiction of the first duty's start station.
    """

    if len(path) < 6:
        return False  # skip very short paths

    # get jurisdiction of the first duty
    start_station_first_duty = path[1].start_station
    start_jurisdictions = get_jurisdiction_groups(start_station_first_duty)

    # Extract times
    start_times = [s.start_time for s in path[1:-1]]
    end_times = [s.end_time for s in path[1:-1]]

    gaps = [
        start_times[i + 1] - end_times[i]
        for i in range(len(start_times) - 1)
    ]

    BREAK_STATIONS = {"KKDA", "PVGW"}  # Breaks allowed only here
    CUMULATIVE_BREAKS_DURATION = 120    # cumulative breaks should be less than 120 min

    breaks = []
    has_break_in_same_jurisdiction = False

    for i, gap in enumerate(gaps):
        station = path[1 + i].end_station  # station where break occurs

        if station in BREAK_STATIONS and gap >= 30:
            breaks.append(gap)

            # check jurisdiction of this break station
            break_jurisdictions = get_jurisdiction_groups(station)
            if not start_jurisdictions.isdisjoint(break_jurisdictions):
                has_break_in_same_jurisdiction = True

    if not breaks:
        return False

    # Ensure at least one break is in same jurisdiction
    if not has_break_in_same_jurisdiction:
        return False

    # Case 1: single break >=50 and <120
    if len(breaks) == 1 and 50 <= breaks[0] < CUMULATIVE_BREAKS_DURATION:
        return True

    # Case 2: at least one >=30 and one >=50, cumulative <120
    has_30 = any(30 <= b < 50 for b in breaks)
    has_50 = any(50 <= b < CUMULATIVE_BREAKS_DURATION for b in breaks)
    total_break = sum(breaks)

    if has_30 and has_50 and total_break < CUMULATIVE_BREAKS_DURATION:
        return True

    return False


## Stack

## Old

In [18]:
def stack_based_all_paths(
    G, start_service, end_service,
    driving_time_limit=360, duty_time_limit=445,
    max_paths=int
):
    """
    Enumerates all feasible paths using stack-based DFS.
    Driving time includes:
    - Service time
    - Gap between services if < 30 minutes
    """
    stack = [(start_service, [start_service], 0)]             # node, path_so_far, driving time
    n = 0

    while stack:
        if n >= max_paths:
            break

        current_service, path, total_driving_time = stack.pop()

        for neighbor_id in G.successors(current_service.service_id):
            neighbor = G.nodes[neighbor_id]["data"]
            service_time = neighbor.service_time or 0

            # Calculate gap safely
            last_service = path[-1]
            if neighbor.start_time is not None and last_service.end_time is not None:
                gap = neighbor.start_time - last_service.end_time
            else:
                gap = 0

            # Driving time = service_time + (gap if <30)
            new_total_driving_time = total_driving_time + service_time
            if gap < 30:
                new_total_driving_time += gap

            new_path = path + [neighbor]

            # Always check acceptability
            if is_path_acceptable(
                new_path, end_service,
                new_total_driving_time, driving_time_limit,
                duty_time_limit
            ):
                if neighbor.service_id == end_service.service_id:
                    if is_final_path_valid(new_path):
                        n += 1
                else:
                    stack.append((neighbor, new_path, new_total_driving_time))

    print(f"Total valid paths found: {n}")
    return n


## Printing formatted paths 

In [19]:
# def stack_based_all_paths(
#     G, start_service, end_service,
#     driving_time_limit=360, duty_time_limit=445,
#     max_paths=int
# ):
#     """
#     Enumerates all feasible paths using stack-based DFS.
#     """
#     stack = [(start_service, [start_service], 0)]
#     valid_paths = []
#     n=0

#     while stack:
#         if n >= max_paths :
#         # if len(valid_paths) >= max_paths:
#         #     print(f"Stopped after reaching {max_paths} valid paths.")
#           break

#         current_service, path, total_driving_time = stack.pop()

#         for neighbor_id in G.successors(current_service.service_id):
#             neighbor = G.nodes[neighbor_id]["data"]
#             service_time = neighbor.service_time or 0

#             new_total_driving_time = total_driving_time + service_time
#             new_path = path + [neighbor]

#             if is_path_acceptable(
#                 new_path, end_service,
#                 new_total_driving_time, driving_time_limit,
#                 duty_time_limit
#             ):
#                 if neighbor.service_id == end_service.service_id:
#                     if is_final_path_valid(new_path):
#                         valid_paths.append(new_path)
#                         n=n+1

#                         # Print path as [S, 34, 45, ..., T]
#                         formatted_path = [
#                             s.service_id
#                             for s in new_path
#                         ]
#                         print(formatted_path)
#                 else:
#                     stack.append((neighbor, new_path, new_total_driving_time))

#     print(f"Total valid paths found: {len(valid_paths)}")
#     #print(f"Total valid paths found: {n}")
#     return valid_paths
#     #return n


## Updated one

In [20]:
# def stack_based_all_paths(
#     G, start_service, end_service,
#     driving_time_limit=360, duty_time_limit=445,
#     max_paths=int
# ):
#     """
#     Enumerates all feasible paths using stack-based DFS.
#     """
#     stack = [(start_service, [start_service], 0)]
#     n = 0

#     while stack:
#         if n >= max_paths:
#             break

#         current_service, path, total_driving_time = stack.pop()

#         for neighbor_id in G.successors(current_service.service_id):
#             neighbor = G.nodes[neighbor_id]["data"]
#             service_time = neighbor.service_time or 0

#             new_total_driving_time = total_driving_time + service_time
#             new_path = path + [neighbor]

#             # === First handle short paths (skip heavy checks) ===
#             if len(new_path) < 5:
#                 stack.append((neighbor, new_path, new_total_driving_time))
#                 continue

#             # === Otherwise, run the usual acceptability checks ===
#             if is_path_acceptable(
#                 new_path, end_service,
#                 new_total_driving_time, driving_time_limit,
#                 duty_time_limit
#             ):
#                 if neighbor.service_id == end_service.service_id:
#                     if is_final_path_valid(new_path):
#                         n += 1
#                 else:
#                     stack.append((neighbor, new_path, new_total_driving_time))

#     print(f"Total valid paths found: {n}")
#     return n


In [21]:
# Start total timer
total_start_time = time.time()

# ================================
#  Load CSV services
# ================================
start_time = time.time()
TIMETABLE_FILE = "C:/Users/srupe/Downloads/crew/mainLoop_rupesh.csv"
services = load_services(TIMETABLE_FILE)
end_time = time.time()
print(f"Loaded {len(services)} services in {end_time - start_time:.3f} seconds")

# ================================
# Build graph with Service objects
# ================================
start_time = time.time()
G = build_graph(services, start_service, end_service)
end_time = time.time()
print(f"Built graph with {len(G.nodes)} nodes and {len(G.edges)} edges in {end_time - start_time:.3f} seconds")

# ================================
# Call stack-based path finder
# ================================
start_time = time.time()
valid_paths_count = stack_based_all_paths(
    G,
    start_service=start_service,
    end_service=end_service,
    driving_time_limit=360,
    duty_time_limit=445,
    max_paths=10000
)
end_time = time.time()
print(f"Found {valid_paths_count} valid paths in {end_time - start_time:.3f} seconds")

# ================================
# Total time
# ================================
total_end_time = time.time()
print(f"[Total Time] {total_end_time - total_start_time:.3f} seconds")




Loaded 794 services in 0.077 seconds
Built graph with 796 nodes and 22953 edges in 0.151 seconds
Total valid paths found: 10000
Found 10000 valid paths in 1.087 seconds
[Total Time] 1.316 seconds


In [ ]:
# import cProfile, pstats

# cProfile.run("stack_based_all_paths(G, start_service, end_service, 360, 445, 10000)", "prof")

# stats = pstats.Stats("prof")
# stats.strip_dirs().sort_stats("cumtime").print_stats(20)


Total valid paths found: 10000
Sun Sep 14 19:13:07 2025    prof

         5654751 function calls in 4.088 seconds

   Ordered by: cumulative time
   List reduced from 40 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.088    4.088 {built-in method builtins.exec}
        1    0.000    0.000    4.088    4.088 <string>:1(<module>)
        1    0.742    0.742    4.088    4.088 1451060194.py:1(stack_based_all_paths)
   176540    0.241    0.000    1.960    0.000 3432717218.py:1(is_final_path_valid)
    82981    0.585    0.000    1.227    0.000 3432717218.py:38(has_required_breaks)
   358790    0.996    0.000    1.039    0.000 3375836363.py:1(is_path_acceptable)
   599501    0.410    0.000    0.768    0.000 3432717218.py:26(get_jurisdiction_groups)
   599501    0.284    0.000    0.284    0.000 3432717218.py:31(<setcomp>)
   358790    0.147    0.000    0.200    0.000 reportviews.py:190(__getitem__)
   12015

In [23]:
# print(stack)

In [24]:
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())


print("\nNodes:")
print(list(G.nodes()))

print("\nEdges:")
print(list(G.edges()))


Number of nodes: 796
Number of edges: 22953

Nodes:
['727', '377', '10', '11', '720', '12', '13', '721', '378', '14', '15', '379', '16', '729', '728', '17', '18', '730', '731', '380', '19', '20', '381', '732', '722', '733', '21', '22', '382', '734', '735', '23', '24', '383', '736', '737', '384', '25', '26', '27', '738', '570', '28', '29', '385', '30', '571', '386', '739', '31', '32', '33', '572', '740', '387', '573', '34', '35', '36', '574', '388', '37', '389', '38', '575', '39', '40', '576', '390', '41', '577', '42', '43', '578', '44', '391', '579', '392', '45', '46', '47', '580', '393', '48', '49', '581', '394', '51', '50', '582', '395', '52', '53', '583', '396', '54', '55', '584', '397', '56', '58', '585', '57', '398', '59', '60', '586', '399', '61', '587', '62', '400', '63', '588', '65', '64', '401', '66', '402', '67', '589', '68', '403', '590', '69', '404', '591', '70', '71', '405', '72', '406', '592', '73', '407', '74', '75', '593', '409', '408', '76', '77', '594', '410', '78', '

In [25]:
# # Call stack-based path finder using Service objects
# valid_paths_count = stack_based_all_paths(
#     G,
#     start_service=start_service,
#     end_service=end_service,
#     driving_time_limit=360,
#     duty_time_limit=445,
#     max_paths=1000
# )

# #print("Total valid paths found:", valid_paths_count)


In [26]:
# formatted_path = [s.service_id if s.service_id in ['S','T'] else s.service_time for s in path]
